## Station Location Features

In [1]:
import pandas as pd
import Useful_functions_for_stations
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap


In [48]:
bluebikes = pd.read_csv('C:/Users/lauri/PycharmProjects/DSML_Data/bluebikes_idle_time_v4.csv')
bluebikes

,start_time,end_time,start_station_id,end_station_id,bike_id,weekday,idle_time
0,2017-01-01 00:06:58,2017-01-01 00:12:49,67,139,644,6,0
1,2017-01-01 00:13:16,2017-01-01 00:28:07,36,10,230,6,0
2,2017-01-01 00:16:17,2017-01-01 00:44:10,36,9,980,6,0
3,2017-01-01 00:21:22,2017-01-01 00:33:50,46,19,1834,6,0
4,2017-01-01 00:30:06,2017-01-01 00:40:28,10,8,230,6,0 days 00:01:59
...,...,...,...,...,...,...,...
3081456,2018-12-31 23:05:54,2018-12-31 23:08:31,43,40,3050,0,0 days 05:03:27
3081457,2018-12-31 23:31:50,2018-12-31 23:39:14,107,90,2673,0,0 days 06:00:40
3081458,2018-12-31 23:48:46,2018-12-31 23:56:55,19,179,4440,0,0 days 06:07:52
3081459,2018-12-31 23:54:16,2018-12-31 23:57:17,352,36,3605,0,0 days 06:59:37


In [31]:
bluebikes_stations = pd.read_csv('C:/Users/lauri/PycharmProjects/DSML_Data/bluebikes_stations.csv')
bluebikes_stations_dist = bluebikes_stations.copy()
bluebikes_stations.head()

,station_id,lat,lon
0,67,42.358100,-71.093198
1,139,42.361780,-71.108100
2,36,42.349792,-71.077338
3,10,42.350406,-71.108279
4,9,42.351187,-71.115852


Density-based feature: Calculate density of stations within a certain radius around each station. High density might imply high competition which could lead to higher idle times.

In [11]:
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic

In [38]:
geodata = gpd.GeoDataFrame(bluebikes_stations, geometry = gpd.points_from_xy(bluebikes_stations.lon, bluebikes_stations.lat))

geodata.set_crs(epsg=4326, inplace=True)

geodata = geodata.to_crs(epsg=3857)

buffer_radius = 1000 # = 1000 meter

geodata['buffer'] = geodata['geometry'].buffer(buffer_radius)
# Calculate number of stations within the radius around a station
geodata['station_density'] = geodata.apply(lambda row: geodata.within(row['buffer']).sum() - 1, axis=1)
# Convert station_density to per square km
geodata['station_density'] = geodata['station_density'] / (3.1416* (buffer_radius/1000)**2)



In [9]:
geodata

,station_id,lat,lon,geometry,buffer,station_density
0,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",3
1,139,42.361780,-71.108100,POINT (-7915717.430 5215327.402),"POLYGON ((-7914717.430 5215327.402, -7914722.2...",8
2,36,42.349792,-71.077338,POINT (-7912293.113 5213521.392),"POLYGON ((-7911293.113 5213521.392, -7911297.9...",9
3,10,42.350406,-71.108279,POINT (-7915737.409 5213613.945),"POLYGON ((-7914737.409 5213613.945, -7914742.2...",5
4,9,42.351187,-71.115852,POINT (-7916580.435 5213731.525),"POLYGON ((-7915580.435 5213731.525, -7915585.2...",4
...,...,...,...,...,...,...
308,229,42.345033,-71.096649,POINT (-7914442.783 5212804.682),"POLYGON ((-7913442.783 5212804.682, -7913447.5...",11
309,374,42.356683,-71.061666,POINT (-7910548.525 5214559.533),"POLYGON ((-7909548.525 5214559.533, -7909553.3...",14
310,376,42.360274,-71.128525,POINT (-7917991.131 5215100.408),"POLYGON ((-7916991.131 5215100.408, -7916995.9...",2
311,377,42.379273,-71.103419,POINT (-7915196.400 5217963.142),"POLYGON ((-7914196.400 5217963.142, -7914201.2...",7


Distance-based feature: There are important locations in Boston like the City_center,the universities or popular train stations. Bike stations closer to those locations might be used more frequently which could lead to lower idle times

In [32]:
# Important locations in Boston
locations = {
    'city_center': (42.360082, -71.058880), # Boston City Center
    'boston_university': (42.350503, -71.105399), # Boston University
    'fenway_park': (42.346676, -71.097218), # Fenway Park
    'harvard_university': (42.373611, -71.109733), # Harvard University
    'mit': (42.360091, -71.094160), # Massachusetts Institute of Technology
    'boston_common': (42.355443, -71.066163), # Boston Common: public park in downtown
    'boston_public_garden': (42.353865, -71.071342), # Boston Public Garden
    'newbury_street': (42.349846, -71.083415), # Newbury Street (shopping district)
    'south_station': (42.352175, -71.055242), # South Station (major train/subway station)
    'north_station': (42.365577, -71.061245), # North Station (major train/subway station)
    'back_bay_station': (42.347351, -71.075187), # Back Bay Station (major train/subway station)
}
# Calculate distance to each location
for loc_name, loc_coord in locations.items():
    bluebikes_stations_dist[loc_name + '_distance'] = bluebikes_stations_dist.apply(lambda row: geodesic((row['lat'], row['lon']), loc_coord).km, axis=1)

bluebikes_stations_dist

,station_id,lat,lon,city_center_distance,boston_university_distance,fenway_park_distance,harvard_university_distance,mit_distance,boston_common_distance,boston_public_garden_distance,newbury_street_distance,south_station_distance,north_station_distance,back_bay_station_distance
0,67,42.358100,-71.093198,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,1.220776,3.195646,2.760200,1.904668
1,139,42.361780,-71.108100,4.059136,1.272305,1.902310,1.321014,1.163583,3.525886,3.153329,2.427624,4.483532,3.882721,3.149924
2,36,42.349792,-71.077338,1.902449,2.313407,1.674192,3.757982,1.797120,1.114398,0.669948,0.500724,1.839755,2.198291,0.323908
3,10,42.350406,-71.108279,4.209433,0.237541,1.001153,2.580400,1.584456,3.514814,3.067487,2.049614,4.374321,4.225473,2.747706
4,9,42.351187,-71.115852,4.796679,0.864608,1.615064,2.541403,2.042611,4.121133,3.679314,2.676779,4.995049,4.774306,3.377621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,229,42.345033,-71.096649,3.532434,0.942848,0.188411,3.352443,1.685154,2.765277,2.304460,1.214474,3.502844,3.703488,1.787130
309,374,42.356683,-71.061666,0.441839,3.667965,3.133033,4.383297,2.703585,0.395249,0.856422,1.946195,0.728667,0.988519,1.521709
310,376,42.360274,-71.128525,5.737488,2.192714,2.989070,2.142643,2.831087,5.165596,4.764520,3.892831,6.104186,5.573593,4.622995
311,377,42.379273,-71.103419,4.243053,3.199969,3.656769,0.816101,2.263149,4.052775,3.866193,3.660657,4.980976,3.792258,4.240577


Stations with lowest distance to north station: 22, 192 59 35 64 48 157 81 365 302 43 151 374 54 23 218 20 7 24 49

In [49]:
# Convert idle_time to timedelta type
bluebikes['idle_time'] = pd.to_timedelta(bluebikes['idle_time'])

# Convert idle_time to minutes
bluebikes['idle_time_hours'] = bluebikes['idle_time'].dt.total_seconds() / 3600

# Calculate total idle time per start station
total_idle_time = bluebikes.groupby('start_station_id')['idle_time_hours'].mean()

total_idle_time

start_station_id
1      18.716289
3       1.887311
4       1.802574
5       0.926074
6       0.656744
         ...    
373    74.029383
374     1.424251
376    10.969248
377     4.868800
378     6.520556
Name: idle_time_hours, Length: 313, dtype: float64

Merge the two tables and compare the idle times with the calculated distances.

In [45]:
merge = pd.merge(bluebikes_stations_dist, total_idle_time, left_on='station_id', right_on='start_station_id', how='inner')
merge

,station_id,lat,lon,city_center_distance,boston_university_distance,fenway_park_distance,harvard_university_distance,mit_distance,boston_common_distance,boston_public_garden_distance,newbury_street_distance,south_station_distance,north_station_distance,back_bay_station_distance,idle_time_hours
0,67,42.358100,-71.093198,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,1.220776,3.195646,2.760200,1.904668,17818.559444
1,139,42.361780,-71.108100,4.059136,1.272305,1.902310,1.321014,1.163583,3.525886,3.153329,2.427624,4.483532,3.882721,3.149924,40969.531111
2,36,42.349792,-71.077338,1.902449,2.313407,1.674192,3.757982,1.797120,1.114398,0.669948,0.500724,1.839755,2.198291,0.323908,14891.648056
3,10,42.350406,-71.108279,4.209433,0.237541,1.001153,2.580400,1.584456,3.514814,3.067487,2.049614,4.374321,4.225473,2.747706,12890.978889
4,9,42.351187,-71.115852,4.796679,0.864608,1.615064,2.541403,2.042611,4.121133,3.679314,2.676779,4.995049,4.774306,3.377621,15306.393333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,229,42.345033,-71.096649,3.532434,0.942848,0.188411,3.352443,1.685154,2.765277,2.304460,1.214474,3.502844,3.703488,1.787130,0.000000
309,374,42.356683,-71.061666,0.441839,3.667965,3.133033,4.383297,2.703585,0.395249,0.856422,1.946195,0.728667,0.988519,1.521709,810.398611
310,376,42.360274,-71.128525,5.737488,2.192714,2.989070,2.142643,2.831087,5.165596,4.764520,3.892831,6.104186,5.573593,4.622995,932.386111
311,377,42.379273,-71.103419,4.243053,3.199969,3.656769,0.816101,2.263149,4.052775,3.866193,3.660657,4.980976,3.792258,4.240577,1363.263889


In [46]:
merge.corr()

,station_id,lat,lon,city_center_distance,boston_university_distance,fenway_park_distance,harvard_university_distance,mit_distance,boston_common_distance,boston_public_garden_distance,newbury_street_distance,south_station_distance,north_station_distance,back_bay_station_distance,idle_time_hours
station_id,1.000000,-0.210642,-0.071248,0.333029,0.340180,0.344237,0.303147,0.378826,0.348926,0.360185,0.372364,0.313229,0.332835,0.348444,-0.636788
lat,-0.210642,1.000000,-0.084013,-0.404726,-0.292182,-0.143003,-0.797734,-0.567075,-0.328446,-0.307813,-0.225413,-0.206425,-0.530772,-0.121763,0.255007
lon,-0.071248,-0.084013,1.000000,-0.637454,0.294445,0.078659,0.364481,0.040594,-0.573774,-0.500103,-0.267153,-0.717860,-0.567687,-0.453744,0.064506
city_center_distance,0.333029,-0.404726,-0.637454,1.000000,0.425940,0.549080,0.377494,0.692257,0.983868,0.957719,0.820515,0.970841,0.986164,0.875511,-0.256764
boston_university_distance,0.340180,-0.292182,0.294445,0.425940,1.000000,0.959331,0.746630,0.898708,0.532505,0.616797,0.813441,0.368777,0.450400,0.665062,-0.189567
fenway_park_distance,0.344237,-0.143003,0.078659,0.549080,0.959331,1.000000,0.586137,0.866077,0.664573,0.742956,0.915319,0.539836,0.537143,0.818027,-0.165725
harvard_university_distance,0.303147,-0.797734,0.364481,0.377494,0.746630,0.586137,1.000000,0.856926,0.393943,0.436358,0.517857,0.201830,0.487442,0.341490,-0.264889
mit_distance,0.378826,-0.567075,0.040594,0.692257,0.898708,0.866077,0.856926,1.000000,0.746337,0.795684,0.873757,0.586955,0.742823,0.753706,-0.264944
boston_common_distance,0.348926,-0.328446,-0.573774,0.983868,0.532505,0.664573,0.393943,0.746337,1.000000,0.992779,0.900807,0.972821,0.958567,0.944540,-0.238981
boston_public_garden_distance,0.360185,-0.307813,-0.500103,0.957719,0.616797,0.742956,0.436358,0.795684,0.992779,1.000000,0.945034,0.948045,0.932712,0.970415,-0.232533


Merge station_density data with the idle time to see if they correlate

In [39]:
merge_geodata = pd.merge(geodata, total_idle_time, left_on='station_id', right_on='start_station_id', how='inner')
merge_geodata

,station_id,lat,lon,geometry,buffer,station_density,idle_time_hours
0,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",0.954927,0.185292
1,139,42.361780,-71.108100,POINT (-7915717.430 5215327.402),"POLYGON ((-7914717.430 5215327.402, -7914722.2...",2.546473,2.415799
2,36,42.349792,-71.077338,POINT (-7912293.113 5213521.392),"POLYGON ((-7911293.113 5213521.392, -7911297.9...",2.864782,0.333819
3,10,42.350406,-71.108279,POINT (-7915737.409 5213613.945),"POLYGON ((-7914737.409 5213613.945, -7914742.2...",1.591546,0.610715
4,9,42.351187,-71.115852,POINT (-7916580.435 5213731.525),"POLYGON ((-7915580.435 5213731.525, -7915585.2...",1.273237,0.786233
...,...,...,...,...,...,...,...
308,229,42.345033,-71.096649,POINT (-7914442.783 5212804.682),"POLYGON ((-7913442.783 5212804.682, -7913447.5...",3.501401,0.000000
309,374,42.356683,-71.061666,POINT (-7910548.525 5214559.533),"POLYGON ((-7909548.525 5214559.533, -7909553.3...",4.456328,1.424251
310,376,42.360274,-71.128525,POINT (-7917991.131 5215100.408),"POLYGON ((-7916991.131 5215100.408, -7916995.9...",0.636618,10.969248
311,377,42.379273,-71.103419,POINT (-7915196.400 5217963.142),"POLYGON ((-7914196.400 5217963.142, -7914201.2...",2.228164,4.868800


In [43]:
merge_geodata.corr()

C:\Users\lauri\AppData\Local\Temp\ipykernel_11520\2671976427.py:1: FutureWarning: The default value of numeric_only in GeoDataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  merge_geodata.corr()


,station_id,lat,lon,station_density,idle_time_hours
station_id,1.000000,-0.210642,-0.071248,-0.191791,0.427416
lat,-0.210642,1.000000,-0.084013,0.159667,-0.521051
lon,-0.071248,-0.084013,1.000000,0.239616,0.022001
station_density,-0.191791,0.159667,0.239616,1.000000,-0.328333
idle_time_hours,0.427416,-0.521051,0.022001,-0.328333,1.000000
